## Flask를 사용한 API 인증 및 키 관리
### 1️⃣ Flask 기본 API 코드

In [3]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/api/data', methods=['GET'])
def get_data():
    return jsonify({"message": "Hello, API!"})

if __name__ == '__main__':
    app.run(debug=False)

# http://127.0.0.1:5000/api/data

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Feb/2025 15:51:34] "GET /api/data HTTP/1.1" 200 -


### 2️⃣ API 키를 사용한 인증
- 클라이언트가 보낸 Authorization 헤더에서 API 키를 확인

In [9]:
from flask import Flask, request, jsonify

app = Flask(__name__)

API_KEY = "my_secret_api_key_1234"  #  API 키 설정

# 요청한 API 키를 검증하는 함수
def verify_api_key(request):
    api_key = request.headers.get("Authorization")
    if api_key and api_key == f"Bearer {API_KEY}":
        return True
    return False    
    
@app.route('/api/data', methods=['GET'])
def get_data():
    if not verify_api_key(request):
        return jsonify({"error": "Unauthorized"}), 401  # 인증 실패
        
    return jsonify({"message":  "Authenticated!", "data": [1, 2, 3, 4, 5]})

if __name__ == '__main__':
    app.run(debug=False)

# http://127.0.0.1:5000/api/data

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Feb/2025 16:42:17] "GET /api/data HTTP/1.1" 401 -


### 3️⃣ 쿼리 파라미터로 API 키 인증
- API 키를 헤더가 아니라 URL 파라미터로 전달

In [13]:
from flask import Flask, request, jsonify

app = Flask(__name__)

API_KEY = "my_secret_api_key_1234"  #  API 키 설정

@app.route('/api/data', methods=['GET'])
def get_data():
    api_key = request.args.get("api_key")
    if not api_key or api_key != API_KEY:
        return jsonify({"error": "Unauthorized"}), 401  # 인증 실패
        
    return jsonify({"message":  "Authenticated!", "data": [1, 2, 3, 4, 5]})

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Feb/2025 16:56:10] "GET /api/data?api_key=my_secret_api_key_1234 HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2025 16:56:56] "GET /api/data HTTP/1.1" 401 -
127.0.0.1 - - [06/Feb/2025 16:57:22] "GET /api/data?api_key=my_secret_api_key_1234 HTTP/1.1" 200 -


### 4️⃣ 보안 강화 : Flask-Limiter를 적용한 API 예제

In [15]:
# Flask-Limiter 설치
# !pip install flask_limiter

In [18]:
from flask import Flask, request, jsonify
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address

app = Flask(__name__)

# 요청 제한 설정 (IP 기반으로 1분에 10번 요청 가능)
limiter = Limiter(get_remote_address, app=app, default_limits=["10 per minute"])

@app.route('/api/data', methods=['GET'])
@limiter.limit("5 per minute")  # 특정 라우트에서 별도의 제한 설정 가능 (여기서는 1분에 5번)
def get_data():
    return jsonify({"message": "Success!", "data": [1, 2, 3, 4, 5]})

if __name__ == '__main__':
    app.run(debug=False)

# Web Browser : http://127.0.0.1:5000/api/data?api_key=my_secret_api_key_1234
# 1분 내에 5번을 초과하면 아래 메시지 출력 (429 Too Many Requests)
# Too Many Requests
# 5 per 1 minute

### API 키를 환경 변수에 저장 (코드에 직접 포함하지 않음)

In [21]:
import os
API_KEY = os.getenv("API_KEY", "default_key")

In [23]:
API_KEY

'default_key'

In [25]:
# .env 파일에 API_KEY=my_secret_api_key1234를 저장하고 python-dotenv 패키지로 불러올 수도 있음.
# 추후 실습